In [1]:
import pandas as pd
import random
from collections import defaultdict
import json

In [2]:
data_dir = "/home/singh_shruti/workspace/rep_learning/eval_llms/modelcards/data/"

In [3]:
with open("../sft_modelcard_qacontext.json", "r") as f:
    qa_data_topk_context = json.load(f)

In [4]:
list(qa_data_topk_context.keys())[0:2]

['mbart_1', 'mbart_2']

In [5]:
list(qa_data_topk_context.values())[21]

{'question': 'What are the gaps in previous literature that this paper tries to address?',
 'answer': 'The last two years have seen the rise of Transfer Learning approaches in Natural Language Processing (NLP) with large-scale pre-trained language models becoming a basic tool in many NLP tasks [Devlin et al., 2018, Radford et al., 2019, Liu et al., 2019]. While these models lead to significant improvement, they often have several hundred million parameters and current research1 on pre-trained models indicates that training even larger models still leads to better performances on downstream tasks. \n\nThe trend toward bigger models raises several concerns. First is the environmental cost of exponentially scaling these models’ computational requirements as mentioned in Schwartz et al. [2019], Strubell et al. [2019]. Second, while operating these models on-device in real-time has the potential to enable novel and interesting language processing applications, the growing computational and 

In [6]:
sheet_to_df_map = pd.read_excel(f'{data_dir}/QAData.xlsx', sheet_name=None)
map_mod_topkfile = {}
map_topkfile_mod = {}
uniq_model_names = {x.split("_")[1].lower(): x for x in sheet_to_df_map.keys()}
uniq_pkeys = []
for k in qa_data_topk_context.keys():
    pkey = k.split("_")[0]
    if not pkey in uniq_pkeys:
        uniq_pkeys.append(pkey)
    map_mod_topkfile[uniq_model_names[pkey]] = k
    map_topkfile_mod[k] = uniq_model_names[pkey]

print(uniq_pkeys == list(uniq_model_names.keys()))
print(uniq_pkeys)
print(list(uniq_model_names.keys()))
print(map_mod_topkfile)

True
['mbart', 'distilbert', 'longformer', 't5', 'sparsetransformer', 'ernie', 'pegasus', 'fnet', 'reformer', 'bart', 'roberta', 'transformerxl', 'bigbird', 'mobilebert', 'gpt2', 'electra', 'structbert', 'muril', 'scibert', 'transformer', 'gpt', 'spanbert', 'albert', 'xlnet', 'bert-pli']
['mbart', 'distilbert', 'longformer', 't5', 'sparsetransformer', 'ernie', 'pegasus', 'fnet', 'reformer', 'bart', 'roberta', 'transformerxl', 'bigbird', 'mobilebert', 'gpt2', 'electra', 'structbert', 'muril', 'scibert', 'transformer', 'gpt', 'spanbert', 'albert', 'xlnet', 'bert-pli']
{'22250016_mBART': 'mbart_20', '22250032_DistilBERT': 'distilbert_20', '22210006_Longformer': 'longformer_20', '22120032_T5': 't5_20', '18210007_SparseTransformer': 'sparsetransformer_20', '22250041_ERNIE': 'ernie_20', '21310020_PEGASUS': 'pegasus_20', '22210040_FNet': 'fnet_20', '22270004_Reformer': 'reformer_20', '22210045_BART': 'bart_20', '22120039_RoBERTa': 'roberta_20', '22210029_TransformerXL': 'transformerxl_20', '2

In [9]:
train_dict = []
test_dict  = []
sheet_to_df_map = pd.read_excel(f'{data_dir}/QAData.xlsx', sheet_name=None)
train_ids = random.sample(list(sheet_to_df_map.keys()), 15)
modkey_title_map = {}

for _, mod_key in enumerate(sheet_to_df_map.keys()):
    all_prompts = []
    model_dict_list = []
    # print(f"Processing model {_}: {mod_key}")
    model_df = sheet_to_df_map[mod_key].loc[9:]
    col_name = sheet_to_df_map[mod_key].columns[1]
    ptitle = list(model_df.columns)[1].strip().replace("\n", "")
    if ptitle == "BART":
        ptitle = "BART: Denoising Sequence-to-Sequence Pre-training for Natural Language Generation, Translation, and Comprehension"
    modkey_title_map[mod_key] = ptitle


for k, val in qa_data_topk_context.items():
    pkey = k.split("_")[0]
    que = val['question']
    context = val['topk_data'][0:3]
    title = modkey_title_map[uniq_model_names[pkey]]
    ans = val['answer']
    
    if uniq_model_names[pkey] in train_ids:
        train_dict.append({"pkey": k, "question": que, "answer": ans, "title": title, "context": context})
    else:
        test_dict.append({"pkey": k, "question": que, "answer": ans, "title": title, "context": context})

In [10]:
with open("../../data/modelcard_ft_data/train.jsonl", "w") as fin:
    for l in train_dict:
        json.dump(l, fin)
        fin.write("\n")

with open("../../data/modelcard_ft_data/test.jsonl", "w") as fin:
    for l in test_dict:
        json.dump(l, fin)
        fin.write("\n")

In [17]:
for _, mod_key in enumerate(sheet_to_df_map.keys()):
    if set(model_ques[mod_key]) == set(model_ques["22250016_mBART"]):
        continue
    else:
        print(f"Not matching: {mod_key} and 22250016_mBART")

Not matching: 22210050_GPT2 and 22250016_mBART
Not matching: 22210028_SpanBERT and 22250016_mBART


In [19]:
for q in model_ques["22210028_SpanBERT"]:
    if not q in set(model_ques["22250016_mBART"]):
        print(q)

List the limitations of the model discussed in the paper.?


In [20]:
set(model_ques["22250016_mBART"])

{'Are ablation studies conducted in the paper? If yes, which parameters are included in the ablation study?',
 'Are all details necessary to reproduce the paper are provided in the paper?',
 'Consider the transformer model as base architecture for encoder-decoder (ED) models. Similarly consider BERT and GPT as base for encoder-only (E) and decoder-only (D) architectures. How is the architecture of this paper different from the base architectures like transformer or bert or gpt (depending on ED, E, or D only model respectively)?',
 'Describe the architecture details (whether it is encoder-decoder, encoder only or decoder only framework, number of layers, number of heads, embedding dimension, total parameters). In case multiple models of varying sizes are trained, list the details for all configurations.',
 'Describe the computational resources used to train the model.',
 'Describe the training setup (e.g. learning rate, steps, epochs, optimizer, etc.)',
 'Does the model show any bias, p

In [5]:
model_df.head()

,Paper Title,mBART: Multilingual Denoising Pre-training for Neural Machine Translation,FILL MODEL NAME AND PAPER TITLE IN CELL B1,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,MISC,"For the paper ""[TITLE]"", please answer the following question, with the heading ""Answer"". Also print the lines from the paper which contain the answer after the heading named ""Evidence"". Question: [QUESTION]"
9,What is the main problem statement that is bei...,"For the paper ""mBART: Multilingual Denoising P...",Answer:\n\nThe main problem statement addresse...,"(1) translation language modeling, where the m...",The main problem statement addressed in this p...,mBART is the first method for pre-training a c...,CC,Answer is correct though ecidence has some inc...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,What are the gaps in previous literature that ...,"For the paper ""mBART: Multilingual Denoising P...",Answer:\n\nThis paper tries to address the gap...,show that it outperforms existing methods for ...,This paper tries to address the gaps in previo...,mBART is the first method for pre-training a c...,CC,Answer is correct though part of the evidence ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,What are the main contributions of the paper?,"For the paper ""mBART: Multilingual Denoising P...",The main contributions of the paper are:\n\nTh...,XNLI cross-lingual classification task,The proposal of a new denoising objective for ...,mBART is the first method for pre-training a c...,CC,Contributions part is correct only the one dat...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,Is the model proposed only for a specific doma...,"For the paper ""mBART: Multilingual Denoising P...","No, the model proposed in this paper, mBART, i...",NaN,"No, the model proposed in this paper, mBART, i...",This paper demonstrates that multilingual deno...,CC,Answer part is correct only the one dataset me...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,What datasets and tasks is the model evaluated...,"For the paper ""mBART: Multilingual Denoising P...","The proposed model, mBART, is evaluated on sev...",XNLI cross-lingual classification task,"The proposed model, mBART, is evaluated on sev...",We evaluate performance on two common document...,PC,Some of the datasets are not mentioned in the ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
